# Gathering pathways, resolving gene homologs and preparing to visualize..

In [15]:
import json
import pickle as pkl
from importlib import reload

import pandas as pd
import numpy as np

from owlready2 import *
import networkx as nx
from pyvis.network import Network

In [193]:
pc = get_ontology("../data/PathwayCommons11.reactome.BIOPAX.owl").load()

## Load PANTHER gene orthologs
```
ftp://ftp.pantherdb.org/ortholog/14.1/RefGenomeOrthologs.tar.gz
```

In [150]:
orthologs = pd.read_csv(
  '../data/RefGenomeOrthologs', delimiter='\t', 
  names=['Gene', 'Ortholog', 'Type', 'Common ancestor', 'something else']
)
orthologs.head()

,Gene,Ortholog,Type,Common ancestor,something else
0,ARATH|TAIR=AT4G37920|UniProtKB=Q84WN0,ARATH|TAIR=AT1G36320|UniProtKB=Q9C8X8,P,Embryophyta|Magnoliophyta,PTHR31755
1,HUMAN|HGNC=10663|UniProtKB=O60524,MOUSE|MGI=MGI=1918305|UniProtKB=Q8CCP0,LDO,Euarchontoglires,PTHR15239
2,HUMAN|HGNC=10663|UniProtKB=O60524,RAT|Ensembl=ENSRNOG00000056128|UniProtKB=A0A0G...,LDO,Euarchontoglires,PTHR15239
3,HUMAN|HGNC=10663|UniProtKB=O60524,CHICK|Ensembl=ENSGALG00000012263|UniProtKB=F1N8T0,LDO,Amniota,PTHR15239
4,HUMAN|HGNC=10663|UniProtKB=O60524,DANRE|Ensembl=ENSDARG00000102859|UniProtKB=A0A...,LDO,Euteleostomi,PTHR15239


## Build an ortholog -> mouse gene map

In [188]:
def convert_db_name(db):
  if db == 'UniProtKB' or db == 'uniprot knowledgebase':
    return 'uniprot'
  elif db == 'ncbi gene':
    return 'ncbi'
  return db.lower()


def convert_db_id(id):
  if ':' in id:
    return id.split(':')[-1].lower()
  return id.lower()


def make_db_ref(db, id):
  return '{0}:{1}'.format(convert_db_name(db), convert_db_id(id))


try:
  # Load precomputed ortholog gene map
  with open('../data/gene-map-all2mouse', 'rb') as f:
    all2mouse = pkl.load(f)
except:
  # Compute and store the ortholog gene map
  all2mouse = {}
  for _, row in orthologs.iterrows():
    # Keep only the closest homologs
    if row['Type'] != 'LDO':
      continue
    if not row['Ortholog'].startswith('MOUSE|'):
      continue

    dst = [
      make_db_ref(d.split('=')[0], d.split('=')[-1])
      for d in row['Ortholog'].split('|')[1:]
    ]

    for src in row['Gene'].split('|')[1:]:
      src_parts = src.split('=')
      db = src_parts[0]
      id = src_parts[-1]
      all2mouse[make_db_ref(db, id)] = dst

  with open('../data/gene-map-all2mouse', 'wb') as f:
    pkl.dump(all2mouse, f)

## Load mouse gene info from NCBI Gene
```
ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Mus_musculus.gene_info.gz
```

In [177]:
def load_gene_info(path):
  """Load the NCBI Gene file and returns a {gene_name: info} dictionary"""
  gene_info = pd.read_csv(path, delimiter='\t')
  gene_info_map = {}
  for _, row in gene_info.iterrows():
    info = {
      'name': row['Symbol'],
      'type': row['type_of_gene'],
      'desc': row['Other_designations'].split('|')[0],
    }
    ids = [make_db_ref('ncbi', row['GeneID'])]
    for xref in row['dbXrefs'].split('|'):
      xref_parts = xref.split(':')
      db = xref_parts[0]
      id = xref_parts[-1]
      ids.append(make_db_ref(db, id))
    for id in ids:
      gene_info_map[id] = info
#     for s in row['Synonyms'].split('|'):
#       gene_info_map[s] = info
  return gene_info, gene_info_map

gene_info, gene_info_map = load_gene_info('../data/Mus_musculus.gene_info')

Exploring the BIOPAX format here..

In [127]:
Pathway = list(filter(lambda x: x.name == 'Pathway', pc.world.classes()))[0]
Catalysis = list(filter(lambda x: x.name == 'Catalysis', pc.world.classes()))[0]
BiochemicalReaction = list(filter(lambda x: x.name == 'BiochemicalReaction', pc.world.classes()))[0]
for p in Pathway.instances()[100:101]:
  print(p.get_properties())
  print(p.displayName)
  print(p.pathwayComponent[0])
  print(p.pathwayComponent[0].get_properties())
  print(p.pathwayComponent[0].displayName)
  print()
  print(p.pathwayComponent[0].controller)
  print(p.pathwayComponent[0].controller[0].get_properties())
  print(p.pathwayComponent[0].controller[0].displayName)
  print(p.pathwayComponent[0].controller[0].entityReference)
  print(p.pathwayComponent[0].controller[0].entityReference.get_properties())
  print(p.pathwayComponent[0].controller[0].entityReference.organism.displayName)
  print(p.pathwayComponent[0].controller[0].entityReference.organism)
  print()
  print(p.pathwayComponent[0].controlled)
  print(p.pathwayComponent[0].controlled.get_properties())
  print(p.pathwayComponent[0].controlled.displayName)
  print(hasattr(p.pathwayComponent[0].controlled, 'displayName'))
  print(p.pathwayComponent[0].controlled.conversionDirection)
  print()
  print(p.pathwayComponent[0].controlled.right[0].get_properties())
  print(p.pathwayComponent[0].controlled.right[0].displayName)
  print(p.pathwayComponent[0].controlled.right[0].entityReference)
  print(p.pathwayComponent[0].controlled.right[0].entityReference.name)
  print(p.pathwayComponent[0].controlled.right[0].entityReference.standardName)
  print(p.pathwayComponent[0].controlled.right[0].entityReference.displayName)
  print(p.pathwayComponent[0].controlled.right[0].entityReference.xref[0].get_properties())
  print()
#   print(p.pathwayComponent)

#   print(p.xref)
#   print(p.xref[0].get_properties())
#   print(p.xref[0].db)

{biopax-level3.name, biopax-level3.pathwayComponent, biopax-level3.xref, biopax-level3.displayName, biopax-level3.dataSource, biopax-level3.standardName, biopax-level3.availability}
Pyrimidine metabolism
.Catalysis_26a2f9367047f173df314be6d019b0cf
{biopax-level3.controller, biopax-level3.comment, biopax-level3.displayName, biopax-level3.dataSource, biopax-level3.controlled}
rn:R02326_Catalysis

[.Protein_807db7d05990fd3bf31ab2a07e82fc60, .Protein_7acd51a37a5a7bc30996301aca90d817]
{biopax-level3.name, biopax-level3.comment, biopax-level3.displayName, biopax-level3.dataSource, biopax-level3.standardName, biopax-level3.entityReference}
NME
uniprot.P56597
{biopax-level3.name, biopax-level3.comment, biopax-level3.xref, biopax-level3.organism, biopax-level3.displayName, biopax-level3.standardName}
Homo sapiens (human)
taxonomy.9606

.BiochemicalReaction_6c699bd73dd15be0db5af66fbe386eb0
{biopax-level3.conversionDirection, biopax-level3.eCNumber, biopax-level3.participantStoichiometry, biopax-

In [189]:
print(json.dumps(all2mouse, indent=2)[:400])

{
  "hgnc:10663": [
    "mgi:1918305",
    "uniprot:q8ccp0"
  ],
  "uniprot:o60524": [
    "mgi:1918305",
    "uniprot:q8ccp0"
  ],
  "hgnc:20854": [
    "mgi:1914066",
    "uniprot:q9d305"
  ],
  "uniprot:q9h0w7": [
    "mgi:1914066",
    "uniprot:q9d305"
  ],
  "hgnc:18049": [
    "mgi:2152200",
    "uniprot:q91va0"
  ],
  "uniprot:q08ah1": [
    "mgi:2152200",
    "uniprot:q91va0"
  ],
  "hgnc:


## Parse BIOPAX into a more actionable form
Extracting pathways and resolving chemical entities

In [192]:
def parse_molecule(m):
  res = {
    'type': 'molecule',
    'name': getattr(m, 'displayName', m.name),
  }
  if hasattr(m, 'entityReference') and m.entityReference is not None:
    e = m.entityReference
    res['entityReference'] = {
      'name': getattr(e, 'displayName', e.name),
    }
    if hasattr(e, 'xref') and e.xref is not None:
      for x in e.xref:
        conv_id = '{0}:{1}'.format(convert_db_name(x.db), convert_db_id(x.id))
        if conv_id in all2mouse:
          for db_ref in all2mouse[conv_id]:
            if db_ref in gene_info_map:
              res['entityReference']['gene'] = gene_info_map[db_ref]
#               print(gene_info_map[db_ref])
#           print(all2mouse[conv_id])
        # TODO: resolve small molecules
        if (x.db == 'ncbi gene') \
          or ( \
            x.db == 'ensembl' \
            and ( \
              'xref' not in res['entityReference'] \
              or res['entityReference']['xref']['db'] != 'ncbi gene' \
            ) \
          ) \
          or ( \
            x.db in ['chebi'] \
            and 'xref' not in res['entityReference'] \
          ):
          # NCBI Gene has the top priority and the Ensemlb goes next
          res['entityReference']['xref'] = {
            'id': x.id,
            'db': x.db,
          }
#       res['entityReference']['xref'] = [
#         {
#           'id': x.id,
#           'db': x.db,
# #           'relationshipType': [
# #              r.term
# #             for r in getattr(x, 'relationshipType', [])
# #           ],
#         }
#         for x in e.xref
#       ]
#     if hasattr(e, 'organism') and e.organism is not None:
#       o = e.organism
#       res['entityReference']['organism'] = {
#         'name': getattr(o, 'displayName', o.name),
#       }
#       if 'Homo' not in res['entityReference']['organism']['name']:
#         print(res['entityReference']['organism']['name'])
  return res


def parse_reaction(r):
  res = {
    'type': 'reaction',
    'name': getattr(r, 'displayName', r.name),
    'comment': getattr(r, 'comment', None),
    'conversionDirection': getattr(r, 'conversionDirection', None),
  }
  if hasattr(r, 'left') and r.left is not None:
    res['left'] = [parse_molecule(c) for _, c in enumerate(r.left)]
  if hasattr(r, 'right') and r.right is not None:
    res['right'] = [parse_molecule(c) for _, c in enumerate(r.left)]
  return res


def parse_catalysis(c):
  res = {
    'type': 'catalysis',
    'name': getattr(c, 'displayName', c.name),
    'comment': getattr(c, 'comment', None),
  }
  if hasattr(c, 'controller') and c.controller is not None:
    res['controller'] = [parse_molecule(cc) for cc in c.controller]
  if hasattr(c, 'controlled') and c.controlled is not None:
    res['controlled'] = parse_reaction(c.controlled)
  return res


def parse_pathway(p):
  res = {
    'type': 'pathway',
    'name': getattr(p, 'displayName', p.name),
    'comment': getattr(p, 'comment', None),
  }
  if hasattr(p, 'pathwayComponent') and p.pathwayComponent is not None:
    res['pathwayComponent'] = []
    for c in p.pathwayComponent:
      if isinstance(c, Catalysis):
        res['pathwayComponent'].append(parse_catalysis(c))
      else:
        res['pathwayComponent'].append(parse_reaction(c))
  return res


# print(list(pc.world.properties()))

for p in Pathway.instances()[100:101]:
  pathway = parse_pathway(p)
#   print(json.dumps(res, indent=2)[:100000])

In [206]:
# g = nx.DiGraph()
g = Network(notebook=True, width='100%')

n = 0
nr = 0
for r in pathway['pathwayComponent']:
  g.add_node(r['name'])
  rr = r
  if r['type'] == 'catalysis':
    for m in r['controller']:
      if 'gene' in m['entityReference']:
        nr += 1
      n += 1
      g.add_node(m['name'])
      g.add_edge(m['name'], r['name'])
    rr = r['controlled']
  # TODO: conversion direction
  for m in rr['left']:
    if 'gene' in m['entityReference']:
      nr += 1
    n += 1
    g.add_node(m['name'])
    g.add_edge(m['name'], r['name'])
  for m in rr['right']:
    if 'gene' in m['entityReference']:
      nr += 1
    n += 1
    g.add_node(m['name'])
    g.add_edge(r['name'], m['name'])

# nx.draw(g)

# net = Network(notebook=True, width='100%')
# net.from_nx(g)
# net.enable_physics(True)
# net.show("pyvis.html")

print(n)
print(nr)

843
71


In [1]:
# import xml.etree.ElementTree as ET
from lxml import etree

In [13]:
mouse_gene2uniprot = {}
with open('../data/uniprot_sprot.xml', 'rb') as f:
  context = etree.iterparse(
    '../data/uniprot_sprot.xml', 
    events=("end",), 
    tag="{http://uniprot.org/uniprot}entry"
  )
  for action, elem in context:
    if action == 'end':
#       print("%s: %s" % (action, elem.tag))
      organism = elem.findall('{http://uniprot.org/uniprot}organism')[0] \
            .findall('{http://uniprot.org/uniprot}name')[0].text.strip()
      if not 'mus musculus' in organism.lower():
        # Memory cleanup
        elem.clear()
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
        continue
      accession = elem.findall('{http://uniprot.org/uniprot}accession')[0].text.strip()
      if len(elem.findall('{http://uniprot.org/uniprot}gene')) < 1:
        continue
      gene_name = elem.findall('{http://uniprot.org/uniprot}gene')[0] \
            .findall('{http://uniprot.org/uniprot}name')[0].text.strip()
      mouse_gene2uniprot[gene_name] = accession
      
      # Memory cleanup
      elem.clear()
      for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
              
#       print('Frog virus 3' in organism)
#       print(elem.xpath('.//name', namespaces=context.nsmap))
#       print(etree.tostring(elem))
#       break

In [16]:
len(mouse_gene2uniprot)

16628

In [23]:
results_merged = pd.read_csv('./results/merged.csv')
daphnia2mouse = pd.read_csv('../data/MetaPhOrs-daphnia2mouse.csv')
daphnia2mouse["daphnia"] = daphnia2mouse["Daphnia pulex"].map(lambda g: g.replace('GO', '').strip())
daphnia2mouse["mouse"] = daphnia2mouse["Mus musculus"].map(lambda g: g.replace('GO', '').strip())
daphnia2mouse.head()

,Lineage,Daphnia pulex,Mus musculus,CS,EL,Trees,phylome,ensembl,orthomcl,treefam,eggnog,hogenom,daphnia,mouse
0,NaN,GO E9HRC7,GO Q922V4,1.000,3,7,NaN,NaN,1.00/5,NaN,NaN,NaN,E9HRC7,Q922V4
1,NaN,GO E9GF52,GO Q3U3J3,0.714,3,7,NaN,NaN,0.60/5,NaN,NaN,NaN,E9GF52,Q3U3J3
2,NaN,GO E9HAK4,GO Q3U1N2,1.000,3,4,NaN,NaN,1.00/2,NaN,NaN,NaN,E9HAK4,Q3U1N2
3,NaN,GO E9G2Y1,GO S4R1P5,1.000,1,1,NaN,NaN,NaN,NaN,NaN,NaN,E9G2Y1,S4R1P5
4,NaN,GO E9FSN2,GO Q8CFE4,1.000,3,7,NaN,NaN,1.00/5,NaN,NaN,NaN,E9FSN2,Q8CFE4


In [44]:
for index, row in results_merged.iterrows():
  if not row['gene'] in mouse_gene2uniprot:
    continue
  gene_name = mouse_gene2uniprot[row['gene']]
  results_merged.loc[index, 'uniprot_mouse'] = gene_name
  
  homolog_match = daphnia2mouse[daphnia2mouse["mouse"] == gene_name]
  if len(homolog_match) < 1:
    continue
  if homolog_match.iloc[0]["EL"] < 2:
    # Low number of supporting data sources
    continue
  homolog = homolog_match.iloc[0]["daphnia"]
  if homolog.startswith('M!'):
    # Not a proper Uniprot id
    continue
#   print(gene_name)
#   print(homolog)
  results_merged.loc[index, 'uniprot_daphnia'] = homolog

In [45]:
results_merged[results_merged['uniprot_daphnia'].notnull()].head()

,Unnamed: 0,index,dataset,start_age,end_age,sex,tissue,subtissue,cell_ontology_class,gene,fold_change_log2,expression_at_start_age,expression_at_end_age,expression_max_std_dev,p_value,uniprot_mouse,uniprot_daphnia
60,19245,5,droplet,24m,30m,male,Kidney,NaN,NaN,Hnf4a,15.215368,4.094030e-09,1.557900e-04,0.000230,0.000000e+00,P49698,E9H7Y7
64,2428,3,droplet,3m,24m,male,Kidney,NaN,NaN,Hnf4a,-15.065793,1.404470e-04,4.094030e-09,0.000258,7.582662e-73,P49698,E9H7Y7
65,5398,15,facs,3m,18m,female,Liver,NaN,NaN,Map4k1,15.054170,3.732086e-09,1.270061e-04,0.000393,1.709817e-08,P70218,E9G8P2
82,42689,0,facs,3m,24m,male,Brain_Non-Myeloid,Striatum,oligodendrocyte,Ppil2,-14.671002,9.562943e-05,3.664892e-09,0.000178,2.025537e-15,Q9D787,E9H7X4
92,42690,1,facs,3m,24m,male,Brain_Non-Myeloid,Striatum,oligodendrocyte,Aldh6a1,-14.523355,8.632671e-05,3.664892e-09,0.000160,1.266913e-15,Q9EQ20,E9GHJ1


In [46]:
results_merged.to_csv('./results/merged_augmented.csv')